# SQL JOINs

When working with relationial databases, we use JOINs to specify the relationships between our tables. The essential idea is that the JOIN will match the rows in one table with rows in one or more other tables. This ways we are able to query information from multiple tables by using the tables primary and foreign keys.

There are several types of JOINs but the ones you will often find yourself using the most are INNER JOINs and LEFT OUTER JOINs.

<img src='Images/SQL_Joins.svg' alt="joins" width="800" />

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
# Mac or Linux: print files in the Data/ folder
# !ls -la Data

In [ ]:
# Windows: print files in the Data/ folder
# dir Data

To begin, let's start by creating a new `company-joins.db` database (store it in your `Data/` folder) and adding an `employee` table to it.

In [ ]:
# Create a new `company-joins.db` database in the Data/ folder
conn = sqlite3.connect('Data/company-joins.db')

# Create a cursor object
cursor = conn.cursor()

In [ ]:
# Drop the existing table
cursor.execute('DROP TABLE IF EXISTS employee;')

# Create a table schema
cursor.execute("""CREATE TABLE employee (
        id            INTEGER  PRIMARY KEY,
        first_name    TEXT     NOT NULL,
        last_name     TEXT     NOT NULL,
        title         TEXT     NOT NULL,
        salary        INTEGER  NOT NULL,
        manager       INTEGER  NULL
);""")


# Insert the table rows
cursor.execute("INSERT INTO employee VALUES (NULL, 'Rupert', 'Giles', 'CEO', 425000, 1);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Willow', 'Rosenberg', 'Data Scientist', 350000, 1);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Anya', 'Jenkins', 'CFO', 380000, 1);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Xander', 'Harris', 'Building Manager', 120500, 0);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Buffy', 'Summers', 'Security Specialist', 240000, 1);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Cordelia', 'Chase', 'Receptionist', 72000, 0);")

# Foreign Keys

Remember that foreign keys allow us to create links or "joins" between the tables in our database. To see how they work, let's create a new `department` table.

```
CREATE TABLE table_name (
  primary_key_col    INTEGER    PRIMARY KEY,
  column             TEXT       NOT NULL,
  foreign_key_col    INTEGER    NOT NULL,
  FOREIGN KEY(primary_key_col) REFERENCES parent_table(parent_table_primary_key)
);""")
```

In [ ]:
# Drop the existing table
cursor.execute('DROP TABLE IF EXISTS department;')

cursor.execute("""CREATE TABLE department (  
    dept_id      INTEGER    NOT NULL    PRIMARY KEY,  
    dept_name    TEXT       NOT NULL,
    dept_head    INTEGER    NOT NULL,
    FOREIGN KEY(dept_id) REFERENCES employee(id)
);""")

cursor.execute("INSERT INTO department VALUES (1, 'Finance', 3)")
cursor.execute("INSERT INTO department VALUES (2, 'Operations', 4)")
cursor.execute("INSERT INTO department VALUES (3, 'Security', 5)")
cursor.execute("INSERT INTO department VALUES (4, 'Data', 1)")

In [ ]:
# Print new `department` table
cursor.execute("SELECT * FROM department;").fetchall()

# INNER JOIN


<img src="Images/inner_joins.png" alt="INNER JOINs" width="300" />

```
SELECT column_name(s)
FROM tableA
  INNER JOIN tableB 
          ON tableA.column = tableB.column;
```

Here, we'll `INNER JOIN` the `employee` and `department` tables.

In [ ]:
# INNER JOIN the tables
df = pd.read_sql("""
         SELECT *
         FROM employee
           INNER JOIN department
                   ON employee.id = department.dept_head;
     """, conn)

# Print your results
df

See how an `INNER JOIN` returns only the rows where the condition specified in the `ON` evaluates to `true`. In addition, notice that by default the columns from the table in the `FROM` clause are listed first.

In the following example, our `INNER JOIN` returns the same data; however, this time the columns from our `department` table are listed first.

In [ ]:
# INNER JOIN the tables
df = pd.read_sql("""
         SELECT *
         FROM department
           INNER JOIN employee
                   ON department.dept_head = employee.id;
     """, conn)

# Print your results
df

## Selecting Columns

```
SELECT
  tableA.column,
  tableB.column
FROM tableA
  INNER JOIN tableB
          ON tableA.column = tableB.column;
```

If we don't want to return all of our columns, we can use dot notation to only select the ones we want to include.

In [ ]:
# INNER JOIN the tables
df = pd.read_sql("""
         SELECT
           employee.id,
           department.dept_head,
           employee.first_name,
           employee.last_name
         FROM employee
           INNER JOIN department
                   ON employee.id = department.dept_head
         ORDER BY id;
     """, conn)

# Print your results
df

# LEFT JOIN

A `LEFT JOIN` will select all of the columns will return every row in the left table, and if the JOIN condition is not met, will return `NULL` values to fill in the columns from the right table.

<img src="Images/left_joins.png" alt="LEFT JOINs" width="300" />

```
SELECT column_name(s)
FROM tableA
  LEFT JOIN tableB 
          ON tableA.column = tableB.column;
```

Now we'll use a `LEFT JOIN` to pull data from the `employee` and `department` tables.

In [ ]:
# LEFT JOIN the `employee` and `department` tables
df = pd.read_sql_query("""
         SELECT *
         FROM employee
           LEFT JOIN department
                  ON employee.id = department.dept_head
         ORDER BY dept_name;
     """, conn)

# Print your results
df

Our `LEFT JOIN` worked but did you notice that for rows that didn't have a matching value, the missing data was filled in sometimes as `NaN` and sometimes as `None`. 

This happens because `NULL` values in SQLite are stored as `None`. Fortunately, we can use numpy to correct this issue by replacing the `None` values with `NaN`.

In [ ]:
import numpy as np

# Replace the None values
df.replace([None], np.nan, inplace=True)

# Print the results
df

Like we did earlier with INNER JOINs, we can also only select a subset of the column names using dot notation.

In [ ]:
# LEFT JOIN the `employee` and `department` tables
df = pd.read_sql_query("""
         SELECT 
           employee.id,
           department.dept_head,
           employee.first_name,
           employee.last_name
         FROM employee
           LEFT JOIN department
                  ON employee.id = department.dept_head;
     """, conn)

# Replace the None values
df.replace([None], np.nan, inplace=True)

# Print your results
df

Always remember to commit your changes and close the connection to your database!

In [ ]:
# Commit any changes
conn.commit()

# Close the database connection
conn.close()